<a href="https://colab.research.google.com/github/shashi36-github/Travel-planner/blob/main/Travel_Planner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TRAVEL PLANNER


In [ ]:
pip install rake-nltk

In [ ]:
import pandas as pd
import random
import nltk
from rake_nltk import Rake
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('vader_lexicon')

# Load the dataset
dataset = pd.read_csv("/content/travel_cities_dataset_updated.csv")

def preprocess_text(sentence):
    tokens = word_tokenize(sentence.lower())
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return set(lemmatized_tokens)

# Function to extract keywords using RAKE and match with attributes
def extract_keywords(sentence, attributes, n_keywords=5):
    rake = Rake()
    rake.extract_keywords_from_text(sentence)
    keywords = rake.get_ranked_phrases()[:n_keywords]
    preprocessed_keywords = preprocess_text(' '.join(keywords))
    matched_attributes = set()
    for keyword in preprocessed_keywords:
        for attribute in attributes:
            if keyword in attribute.lower():
                matched_attributes.add(attribute)
            else:
                for syn in wordnet.synsets(keyword):
                    for lemma in syn.lemmas():
                        if lemma.name() in attribute.lower():
                            matched_attributes.add(attribute)
                            break
    return matched_attributes

# Function to suggest destinations
def suggest_destinations(matched_attributes):
    matched_destinations = []
    for index, row in dataset.iterrows():
        destination_attributes = set(row['Attributes'].split(','))
        score = len(matched_attributes.intersection(destination_attributes))
        if score > 0:
            matched_destinations.append((row['Destination'], destination_attributes, score))
    matched_destinations.sort(key=lambda x: (x[2], x[2]), reverse=True)
    return matched_destinations[:3]

# Function to analyze sentiment
def analyze_sentiment(feedback):
    sia = SentimentIntensityAnalyzer()
    sentiment_score = sia.polarity_scores(feedback)['compound']
    return sentiment_score

# Function to update attribute scores based on user feedback
def update_attribute_scores(destination, attribute_index, sentiment_score):

    index = dataset.index[dataset['Destination'] == destination].tolist()
    if not index:
        print(f"Destination '{destination}' not found in the dataset.")
        return

    index = index[0]


    attribute_score_column = f"Attribute{attribute_index}_Score"

    print(f"Updating attribute score for {destination}, attribute {attribute_index}, sentiment score {sentiment_score}")


    current_score = dataset.at[index, attribute_score_column]

    # Update the score based on user feedback
    updated_score = float(current_score) + 0.3 * sentiment_score

    print(f"Current score: {current_score}, Updated score: {updated_score}")


    dataset.at[index, attribute_score_column] = str(updated_score)


    dataset.to_csv("travel_cities_dataset_updated.csv", index=False)


def main():
    attributes = dataset['Attributes'].str.split(',').explode().str.strip().unique()

    print("Welcome to the Travel Destination Recommender!")
    print("Please provide some information about your travel preferences.")

    while True:
        user_input = input("Your input: ")


        matched_attributes = extract_keywords(user_input, attributes)


        recommendations = suggest_destinations(matched_attributes)

        if recommendations:
            print("Based on your input, here are some travel destinations:")
            for i, (destination, destination_attributes, score) in enumerate(recommendations, 1):
                print(f"{i}. Destination: {destination}, Attributes: {', '.join(destination_attributes)} (Match Score: {score})")
        else:
            print("Sorry, I couldn't find any suitable travel destinations based on your input.")


        feedback = input("Do any of these recommendations suit your taste? ")


        sentiment_score = analyze_sentiment(feedback)

        # Update attribute scores based on user feedback
        if sentiment_score > 0:
            destination_index = int(input("Enter the index of the destination you liked (1, 2, 3): ")) - 1
            destination, destination_attributes, _ = recommendations[destination_index]
            print(f"Attributes of {destination}:")
            for i, attr in enumerate(destination_attributes, 1):
                print(f"{i}. {attr}")
            selected_attribute_index = int(input("Enter the index of the attribute you liked: ")) - 1
            update_attribute_scores(destination, selected_attribute_index + 1, sentiment_score)
        elif sentiment_score < 0:
            print("Which destination you didn't like?")
            destination_index = int(input()) - 1
            destination, destination_attributes, _ = recommendations[destination_index]
            print(f"Here are the attributes of the destination {destination}:")
            for i, attr in enumerate(destination_attributes, 1):
                print(f"{i}. {attr}")
            attribute_index = int(input("Enter the index of the attribute you didn't like: ")) - 1
            update_attribute_scores(destination, attribute_index + 1, sentiment_score)
        else:
            print("No sentiment detected.")

        # Ask if the user wants to continue or stop
        choice = input("Would you like to continue exploring travel destinations? (yes/no) ").lower()
        if choice == "no":
            print("Thank you for using the Travel Destination Recommender. Have a great day!")
            break
        elif choice != "yes":
            print("Sorry, I didn't understand your response. Let's continue exploring.")
            continue

if __name__ == "__main__":
    main()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Welcome to the Travel Destination Recommender!
Please provide some information about your travel preferences.
Your input: I want to go for boating
Based on your input, here are some travel destinations:
1. Destination: Surat, Attributes: boating, sports, gardens, museums (Match Score: 1)
2. Destination: Nashik, Attributes: beaches, boating, handicrafts (Match Score: 1)
3. Destination: Srinagar, Attributes: cycling, wildlife, dance performances, boating (Match Score: 1)
Do any of these recommendations suit your taste? yes they do
Enter the index of the destination you liked (1, 2, 3): 3
Attributes of Srinagar:
1. cycling
2. wildlife
3. dance performances
4. boating
Enter the index of the attribute you liked: 2
Updating attribute score for Srinagar, attribute 2, sentiment score 0.4019
Current score: 1, Updated score: 1.12057
Would you like to continue exploring travel destinations? (yes/no) no
Thank you for using the Travel Destination Recommender. Have a great day!


In [ ]:
attributes = dataset['Attributes'].str.split(',').explode().str.strip().unique()

# Test cases
test_cases = [
    {"input": "historical landmarks", "expected_keywords": {"historical landmarks"}},
    {"input": "vibrant nightlife", "expected_keywords": {"nightlife"}},
    {"input": "cultural festivals ", "expected_keywords": {"cultural festivals"}},
    {"input": "beautiful beaches", "expected_keywords": {"beaches"}},
    {"input": "handicrafts", "expected_keywords": {"handicrafts"}},
    {"input": "nature photography", "expected_keywords": {"photography"}},
    {"input": "hiking ", "expected_keywords": {"hiking"}},
    {"input": "boating ", "expected_keywords": {"boating"}},
    {"input": "beautiful scenery", "expected_keywords": {"scenery"}},
    {"input": "vibrant nightlife", "expected_keywords": {"nightlife"}},
    {"input": "souvenirs ", "expected_keywords": {"souvenirs"}},
    {"input": "snorkeling", "expected_keywords": {"snorkeling"}},
    {"input": "wildlife", "expected_keywords": {"wildlife"}},
]

# Count of correct extractions
correct_extractions = 0

# Iterate through keyword test cases
for case in test_cases:
    input_sentence = case["input"]
    expected_keywords = case["expected_keywords"]

    # Extract keywords from input sentence
    extracted_keywords = extract_keywords(input_sentence, attributes)

    # Check if the extracted keywords match the expected keywords
    if extracted_keywords == expected_keywords:
        correct_extractions += 1

# Calculate accuracy
total_keyword_test_cases = len(test_cases)
accuracy = (correct_extractions / total_keyword_test_cases) * 100

print(f"Keyword Extraction Accuracy: {accuracy:.2f}%")

Keyword Extraction Accuracy: 92.31%
